In [ ]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def get_dice(rlist):
    df = pd.DataFrame(rlist)
    df1sum = df.sum(axis=0)

    TP1 = df1sum[0]
    FN1 = df1sum[1]
    FP1 = df1sum[2]

    D = (2 * TP1) / (2 * TP1 + FN1 + FP1)

    return D


def dice_agg(dataset):
    p = Path(f'tmp/nnUNet/nnUNet_results/{dataset}')
    train_configs = [pi for pi in p.glob('*')]
    dices1 = []
    dices2 = []
    cols = []
    for train_config in train_configs:
        if '1ep' in str(train_config) or '10ep' in str(train_config):
            continue

        cdice1 = []
        cdice2 = []
        print(f'Calculating for:\n {str(train_config)}')
        cols.append(str(train_config).split('\\')[-1])

        summary_path_list = [str(pi) for pi in train_config.glob('**/summary.json')]

        c1 = []
        c2 = []
        for path in summary_path_list:

            with open(path) as file:
                summary = json.load(file)
            c1_fold = []
            c2_fold = []
            sl = summary['metric_per_case']

            for i in range(len(sl)):
                tp1 = sl[i]['metrics']['1']['TP']
                fn1 = sl[i]['metrics']['1']['FN']
                fp1 = sl[i]['metrics']['1']['FP']

                tp2 = sl[i]['metrics']['2']['TP']
                fn2 = sl[i]['metrics']['2']['FN']
                fp2 = sl[i]['metrics']['2']['FP']
                case1 = [tp1, fn1, fp1]
                case2 = [tp2, fn2, fp2]

                c1_fold.append(case1)
                c2_fold.append(case2)

            c1 += c1_fold
            c2 += c2_fold

            dice1 = get_dice(c1_fold)
            dice2 = get_dice(c2_fold)

            cdice1.append(dice1)
            cdice2.append(dice2)

        dice1 = get_dice(c1)
        dice2 = get_dice(c2)

        # cdice1.append(dice1)
        # cdice2.append(dice2)

        dices1.append(cdice1)
        dices2.append(cdice2)

        print(f' D1: {dice1}\n D2: {dice2}\n mu: {np.mean([dice1, dice2])}')

    df_d1 = pd.DataFrame(dices1).transpose()
    df_d2 = pd.DataFrame(dices2).transpose()
    df_d1.columns = cols
    df_d2.columns = cols
    return df_d1, df_d2

In [ ]:
dataset = 'Dataset101_HNTSMRG'
df1, df2 = dice_agg(dataset)

In [ ]:
dataset = 'Dataset102_HNTSMRG'
df1, df2 = dice_agg(dataset)

In [ ]:
s = 1.2
names = [f'{i:02d}' for i in range(15)]
fig, ax = plt.subplots(1, 2, figsize=(8 * s, 3 * s))

legs = ['1000 epochs', '1250 epochs']
colors = ['tab:green', 'tab:purple']

df1.plot(y=['nnUNetTrainer__nnUNetPlans__3d_fullres', 'nnUNetTrainer_1250epochs__nnUNetPlans__3d_fullres'], kind='bar',
         ax=ax[0], width=.8, color=colors, alpha=.8)
ax[0].set_ylim(.25, 1)
ax[0].set_xticks(list(range(15)), names, rotation=45)
ax[0].legend(labels=legs, loc="upper left", frameon=False)
ax[0].set_ylabel(r"$DSC_{agg}$")
ax[0].set_title("GTVp")
ax[0].set_xlabel("Fold")
ax[0].spines[['right', 'top']].set_visible(False)

df2.plot(y=['nnUNetTrainer__nnUNetPlans__3d_fullres', 'nnUNetTrainer_1250epochs__nnUNetPlans__3d_fullres'], kind='bar',
         ax=ax[1], width=.8, color=colors, alpha=.8)

ax[1].set_ylim(.25, 1)
ax[1].set_xticks(list(range(15)), names, rotation=45)
#ax[1].legend(labels = legs, loc = "lower left")
ax[1].get_legend().remove()
ax[1].set_ylabel(r"$DSC_{agg}$")
ax[1].set_title("GTVn")
ax[1].set_xlabel("Fold")
ax[1].spines[['right', 'top']].set_visible(False)

plt.tight_layout()
plt.savefig('epochs.svg')
plt.show()

In [ ]:
d1000 = np.array(df1['nnUNetTrainer__nnUNetPlans__3d_fullres'])
d1250 = np.array(df1['nnUNetTrainer_1250epochs__nnUNetPlans__3d_fullres'])
np.sum(d1250 > d1000)

In [ ]:
d1000 = np.array(df2['nnUNetTrainer__nnUNetPlans__3d_fullres'])
d1250 = np.array(df2['nnUNetTrainer_1250epochs__nnUNetPlans__3d_fullres'])
np.sum(d1250 > d1000)

In [ ]:
print(df1['nnUNetTrainer__nnUNetPlans__3d_fullres'])
print(df1['nnUNetTrainer_1250epochs__nnUNetPlans__3d_fullres'])